In [483]:
import numpy as np
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
import csv
import os
import pandas as pd
import timeit
import math
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import PercentFormatter
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
from textwrap import wrap
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from mpl_toolkits.mplot3d import proj3d
from itertools import product, combinations
import sys
from tqdm import tqdm
#matplotlib.use('Qt5Agg')
start = timeit.default_timer()

In [503]:
import sys 
sys.path 

['d:\\Projects\\molecules article',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test\\python39.zip',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test\\DLLs',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test\\lib',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test',
 '',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test\\lib\\site-packages',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test\\lib\\site-packages\\win32',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\giorg\\miniconda3\\envs\\algonauts_test\\lib\\site-packages\\Pythonwin']

In [504]:
path_risultati = "./FINALE/"
path_dati = './Dati/'
cartella_e_nome_output = "Replica1"
#path_cartella = path_dati + 'Solo_H2O\\' + cartella_e_nome_output #cartella contenente i files.pdb
path_cartella = path_dati + 'H2O_NaCl\\' + cartella_e_nome_output #cartella contenente i files.pdb
tipo_di_ambiente = "NaCl"
#COSTANTI FONDAMENTALI
dimensioni_scatola = [39.606,39.606,50.]
numero_di_frame = 3000
distanza_max_clusterizzazione = 4
numero_minimo_elementi = 3

In [505]:
def nuovi_indici(vecchi_indici):
    indici_o_nuovi = np.array(vecchi_indici + (vecchi_indici*2))
    indici_h1_nuovi = np.array(indici_o_nuovi + 1)
    indici_h2_nuovi = np.array(indici_o_nuovi + 2)
    return indici_o_nuovi, indici_h1_nuovi, indici_h2_nuovi

def periodicita(L = dimensioni_scatola[0]): #L larghezza della scatola
    total = 0
    for d in range(data_solo_ossigeno.shape[1]):
        pd = pdist(data_solo_ossigeno[:, d].reshape(-1, 1))
        if (d != 2):
            pd[pd > (0.5 * L)] -= L
        total += pd ** 2

    total = np.sqrt(total)
    square = squareform(total)
    return square

def outofbox():
    # TEST PER VALORI DI |X,Y| MAGGIORI DI 19.803 - OVVERO L/2
    data_solo_ossigeno[:, :2] = np.where((data_solo_ossigeno[:, :2] > 19.803) & (((data_solo_ossigeno[:, :2] // 19.803) % 2) == 1), (-(19.803) + (data_solo_ossigeno[:, :2] % 19.803)), data_solo_ossigeno[:, :2])
    data_solo_ossigeno[:, :2] = np.where((data_solo_ossigeno[:, :2] > 19.803) & (((data_solo_ossigeno[:, :2] // 19.803) % 2) == 0), data_solo_ossigeno[:, :2] % 19.803, data_solo_ossigeno[:, :2])
    # test per valori di x,y minori di -20
    data_solo_ossigeno[:, :2] = np.where((data_solo_ossigeno[:, :2] < (-19.803)) & (((data_solo_ossigeno[:, :2] // (-19.803)) % 2) == 1),
                                         [+19.803 + (data_solo_ossigeno[:, :2] % (-19.803))], data_solo_ossigeno[:, :2])
    data_solo_ossigeno[:, :2] = np.where((data_solo_ossigeno[:, :2] < (-19.803)) & (((data_solo_ossigeno[:, :2] // (-19.803)) % 2) == 0), data_solo_ossigeno[:, :2] % (-19.803),
                                         data_solo_ossigeno[:, :2])
    return data_solo_ossigeno


def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [506]:
colonne_nomi=np.array(["0","1","2","3","4","x","y","z"]) #colonne del dataframe .pdb
colonne_elemento_e_coordinate = ["2","x","y","z"]

In [507]:
files_pdb = os.listdir(path_cartella)  #crea una lista contenente tutti i nomi dei file
print(files_pdb)

['Simulazione_Celsius24_Milliatmosfere7.500_5Layers4Mobili_300+300Msteps.pdb', 'Simulazione_Celsius24_Milliatmosfere8.750_5Layers4Mobili_300+300Msteps.pdb', 'Simulazione_Celsius24_Milliatmosfere9.000_5Layers4Mobili_300+300Msteps.pdb']


In [508]:
x = 0
path = path_cartella + f"\{files_pdb[x]}"
print(path)
dataframe = pd.DataFrame()
dataframe = pd.read_csv(path, delim_whitespace=True, names=colonne_nomi, low_memory=False, keep_default_na=False, na_values=[''])
dataframe.head()

./Dati/H2O_NaCl\Replica1\Simulazione_Celsius24_Milliatmosfere7.500_5Layers4Mobili_300+300Msteps.pdb


,0,1,2,3,4,x,y,z
0,REMARK,MODEL,1,NaN,NaN,NaN,NaN,NaN
1,MODEL,1,NaN,NaN,NaN,NaN,NaN,NaN
2,ATOM,1,NA,SLT,1.0,12.330,-18.789,0.0
3,ATOM,1,NA,SLT,1.0,17.988,-18.789,0.0
4,ATOM,1,NA,SLT,1.0,9.501,-15.960,0.0


In [509]:
dataframe.tail()

,0,1,2,3,4,x,y,z
3127818,ATOM,1,OW,ACQ,1.0,14.640,-16.217,8.536
3127819,ATOM,1,HW,ACQ,1.0,13.866,-15.708,8.913
3127820,ATOM,1,HW,ACQ,1.0,14.539,-17.188,8.753
3127821,ENDMDL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3127822,END,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [510]:
troncone_numero_ = "2"
dataframe.loc[1:, "1"] = dataframe.loc[1:, "1"].apply(pd.to_numeric, errors='coerce')
posizione_model = np.append([0], np.where(dataframe.loc[1:, "1"] > 1))
posizione_model += 1
numero_di_linee_totali = file_len(path)

In [511]:
dataframe["2"].value_counts(dropna=False)

NX     1176000
CX     1176000
NA      294000
CL      294000
HW      121214
OW       60607
NaN       6001
1            1
Name: 2, dtype: int64

In [512]:
# crea una nuova colonna chiamata frame piena di NaN
dataframe["modello"] = np.nan
dataframe.head()

,0,1,2,3,4,x,y,z,modello
0,REMARK,MODEL,1,NaN,NaN,NaN,NaN,NaN,NaN
1,MODEL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATOM,1.0,NA,SLT,1.0,12.330,-18.789,0.0,NaN
3,ATOM,1.0,NA,SLT,1.0,17.988,-18.789,0.0,NaN
4,ATOM,1.0,NA,SLT,1.0,9.501,-15.960,0.0,NaN


In [513]:
# crea un dataframe vuoto con le colonne "atom", "x", "y", "z", "frame"
len(posizione_model)

3000

In [514]:
dataframe[posizione_model[-1] : -2]

,0,1,2,3,4,x,y,z,modello
3126789,MODEL,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3126790,ATOM,1.0,NA,SLT,1.0,12.330,-18.789,0.000,NaN
3126791,ATOM,1.0,NA,SLT,1.0,17.988,-18.789,0.000,NaN
3126792,ATOM,1.0,NA,SLT,1.0,9.501,-15.960,0.000,NaN
3126793,ATOM,1.0,NA,SLT,1.0,15.159,-15.960,0.000,NaN
...,...,...,...,...,...,...,...,...,...
3127816,ATOM,1.0,HW,ACQ,1.0,1.914,1.407,-7.618,NaN
3127817,ATOM,1.0,HW,ACQ,1.0,2.660,2.621,-8.415,NaN
3127818,ATOM,1.0,OW,ACQ,1.0,14.640,-16.217,8.536,NaN
3127819,ATOM,1.0,HW,ACQ,1.0,13.866,-15.708,8.913,NaN


In [515]:
for i in range(len(posizione_model) - 1):
    dataframe.loc[posizione_model[i] + 1 : posizione_model[i+1] - 2, 'modello'] = i+1

# assegna il valore numero_di_frame da posizione_model[-1] a indice -2
dataframe.loc[posizione_model[-1] +1 : dataframe.index[-2], 'modello'] = float(numero_di_frame)

In [516]:
dataframe.tail()

,0,1,2,3,4,x,y,z,modello
3127818,ATOM,1.0,OW,ACQ,1.0,14.640,-16.217,8.536,3000.0
3127819,ATOM,1.0,HW,ACQ,1.0,13.866,-15.708,8.913,3000.0
3127820,ATOM,1.0,HW,ACQ,1.0,14.539,-17.188,8.753,3000.0
3127821,ENDMDL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0
3127822,END,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [517]:
start_indices = [posizione_model[i]+1 for i in range(len(posizione_model)-1)] + [posizione_model[-1] + 1]
end_indices = [posizione_model[i+1]-2 for i in range(len(posizione_model)-1)] + [dataframe.index[-3]]
filtered_indices = [idx for start, end in zip(start_indices, end_indices) for idx in range(start, end+1)]
dataframe = dataframe.loc[filtered_indices]
dataframe = dataframe[['2', 'x', 'y', 'z', 'modello']]
# rename columns as 'atom', 'x', 'y', 'z', 'frame'
dataframe.columns = ['atom', 'x', 'y', 'z', 'frame']
# reset index
dataframe.reset_index(drop=True, inplace=True)
dataframe.head()

,atom,x,y,z,frame
0,NA,12.330,-18.789,0.0,1.0
1,NA,17.988,-18.789,0.0,1.0
2,NA,9.501,-15.960,0.0,1.0
3,NA,15.159,-15.960,0.0,1.0
4,NA,12.330,-13.131,0.0,1.0


# Hydrogen 

## Far Distance (0 - 14 A)

In [518]:
# mantain only rows with atom == 'HW'
dataframe_h = dataframe[dataframe['atom'] == 'HW']
dataframe_h.head()

,atom,x,y,z,frame
981,HW,-12.253,17.270,-8.552,1.0
982,HW,-13.136,16.061,-7.900,1.0
984,HW,13.876,-13.601,8.221,1.0
985,HW,12.495,-14.466,8.319,1.0
987,HW,1.492,-1.545,-8.840,1.0


In [519]:
dataframe_h.tail()

,atom,x,y,z,frame
3121814,HW,0.713,4.024,-9.046,3000.0
3121816,HW,1.914,1.407,-7.618,3000.0
3121817,HW,2.660,2.621,-8.415,3000.0
3121819,HW,13.866,-15.708,8.913,3000.0
3121820,HW,14.539,-17.188,8.753,3000.0


In [520]:
dataframe_h["atom"].value_counts(dropna=False)

HW    121214
Name: atom, dtype: int64

Splitting in positive and negative values

In [521]:
dataframe_positive = dataframe_h.loc[dataframe['z'] > 0]
dataframe_negative = dataframe_h.loc[dataframe['z'] < 0]

Effettua calcoli:
1. Trova l'indice di distanza da assegnare a ogni idrogeno
2. elimina gli idrogeni più distanti di 14

In [522]:
def calculate_d(z):
    d_values = np.arange(7.625, 14.125, 0.125)
    for d in d_values:
        if abs(z) <= d:
            return d
    return d_values[-1]

# Aggiunta della colonna "d" nei dataframe
dataframe_positive['d'] = dataframe_positive['z'].apply(calculate_d)
dataframe_negative['d'] = dataframe_negative['z'].apply(calculate_d)
dataframe_positive = dataframe_positive.loc[abs(dataframe_positive['z']) <= 14]
dataframe_negative = dataframe_negative.loc[abs(dataframe_negative['z']) <= 14]

C:\Users\giorg\AppData\Local\Temp\ipykernel_6016\658969078.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_positive['d'] = dataframe_positive['z'].apply(calculate_d)
C:\Users\giorg\AppData\Local\Temp\ipykernel_6016\658969078.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_negative['d'] = dataframe_negative['z'].apply(calculate_d)


In [523]:
dataframe_positive[dataframe_positive['d'] == 14]

,atom,x,y,z,frame,d
291074,HW,-15.020,19.886,13.876,282.0,14.0
2499777,HW,15.135,8.993,13.922,2398.0,14.0


In [524]:
# Raggruppamento per frame e d, calcolo del conteggio
grouped_positive = dataframe_positive.groupby(['frame', 'd']).size().reset_index(name='count')
grouped_negative = dataframe_negative.groupby(['frame', 'd']).size().reset_index(name='count')

In [525]:
grouped_negative.tail()

,frame,d,count
32222,3000.0,8.875,1
32223,3000.0,9.000,3
32224,3000.0,9.125,3
32225,3000.0,9.250,2
32226,3000.0,9.500,1


In [526]:
grouped_positive.head()

,frame,d,count
0,1.0,7.750,1
1,1.0,7.875,1
2,1.0,8.000,1
3,1.0,8.125,1
4,1.0,8.250,1


In [527]:
# Creazione del dataframe di riferimento con tutte le combinazioni di frame e d
frames1 = np.arange(1.0, 3001.0, 1.0)
d_values1 = np.arange(7.625, 14.125, 0.125)
df_reference = pd.DataFrame([(frame, d) for frame in frames1 for d in d_values1], columns=['frame', 'd'])

In [528]:
grouped_negative_fill = pd.merge(grouped_negative, df_reference, how='right', on=['frame', 'd'])
grouped_positive_fill = pd.merge(grouped_positive, df_reference, how='right', on=['frame', 'd'])
grouped_negative_fill['count'].fillna(0, inplace=True)
grouped_positive_fill['count'].fillna(0, inplace=True)

In [529]:
grouped_positive_fill.head()

,frame,d,count
0,1.0,7.625,0.0
1,1.0,7.750,1.0
2,1.0,7.875,1.0
3,1.0,8.000,1.0
4,1.0,8.125,1.0


In [530]:
positive_negative = pd.merge(grouped_positive_fill, grouped_negative_fill, how='inner', on=['frame', 'd'])
positive_negative_grouped = positive_negative.groupby('d')[['count_x', 'count_y']].mean()
positive_negative_grouped

# Rinomina delle colonne del nuovo dataframe
positive_negative_grouped.columns = ['Average N of H (positive z)', 'Average N of H (negative z)']

# Creazione degli indici come valori da 7.625 a 14
d_values = np.arange(7.625, 14.125, 0.125)
# subtract 1.78271213799185 to every value of d_values
d_values -= 5.84228786200815
positive_negative_grouped.index = d_values
positive_negative_grouped

,Average N of H (positive z),Average N of H (negative z)
1.782712,0.605667,0.565667
1.907712,0.766000,0.683333
2.032712,1.113000,1.006333
2.157712,1.382667,1.143667
2.282712,1.416667,1.197667
2.407712,1.346000,1.145667
2.532712,1.269000,1.096333
2.657712,1.321667,1.081333
2.782712,1.400667,1.171333
2.907712,1.528000,1.247333


In [531]:
def calculate_positive_density(positive_density):
    positive_density /= 2
    positive_density *= 18
    positive_density /= 0.196079405
    positive_density /= 6.02214e23
    positive_density *= 1e21
    return positive_density

positive_negative_grouped['Average Density (positive z)'] = positive_negative_grouped['Average N of H (positive z)'].apply(calculate_positive_density)
positive_negative_grouped['Average Density (negative z)'] = positive_negative_grouped['Average N of H (negative z)'].apply(calculate_positive_density)

In [532]:
positive_negative_grouped

,Average N of H (positive z),Average N of H (negative z),Average Density (positive z),Average Density (negative z)
1.782712,0.605667,0.565667,0.046163,0.043114
1.907712,0.766000,0.683333,0.058383,0.052083
2.032712,1.113000,1.006333,0.084831,0.076701
2.157712,1.382667,1.143667,0.105385,0.087168
2.282712,1.416667,1.197667,0.107976,0.091284
2.407712,1.346000,1.145667,0.102590,0.087321
2.532712,1.269000,1.096333,0.096721,0.083561
2.657712,1.321667,1.081333,0.100735,0.082417
2.782712,1.400667,1.171333,0.106757,0.089277
2.907712,1.528000,1.247333,0.116462,0.095070


In [533]:
# salva il dataset come file .csv
positive_negative_grouped.to_csv(files_pdb[x].rstrip(".pdb") + ".csv")

## Near Distance (< 2 A)

In [499]:
# mantain only rows with atom == 'HW'
dataframe_h = dataframe[dataframe['atom'] == 'HW']
dataframe_h.head()

,atom,x,y,z,frame
981,HW,-12.253,17.270,-8.552,1.0
982,HW,-13.136,16.061,-7.900,1.0
984,HW,13.876,-13.601,8.221,1.0
985,HW,12.495,-14.466,8.319,1.0
987,HW,1.492,-1.545,-8.840,1.0


In [500]:
dataframe_h.tail()

,atom,x,y,z,frame
3121814,HW,0.713,4.024,-9.046,3000.0
3121816,HW,1.914,1.407,-7.618,3000.0
3121817,HW,2.660,2.621,-8.415,3000.0
3121819,HW,13.866,-15.708,8.913,3000.0
3121820,HW,14.539,-17.188,8.753,3000.0


In [501]:
dataframe_h["atom"].value_counts(dropna=False)

HW    121214
Name: atom, dtype: int64

Splitting in positive and negative values

In [502]:
dataframe_positive = dataframe_h.loc[dataframe['z'] > 0]
dataframe_negative = dataframe_h.loc[dataframe['z'] < 0]

Effettua calcoli:
1. Trova l'indice di distanza da assegnare a ogni idrogeno
2. elimina gli idrogeni più distanti di 14

In [470]:
def calculate_d(z):
    d_values = np.arange(5.842, 14.125, 0.125)
    for d in d_values:
        if abs(z) <= d:
            return d
    return d_values[-1]

# Aggiunta della colonna "d" nei dataframe
dataframe_positive['d'] = dataframe_positive['z'].apply(calculate_d)
dataframe_negative['d'] = dataframe_negative['z'].apply(calculate_d)
dataframe_positive = dataframe_positive.loc[abs(dataframe_positive['z']) <= 14.]
dataframe_negative = dataframe_negative.loc[abs(dataframe_negative['z']) <= 14.]

C:\Users\giorg\AppData\Local\Temp\ipykernel_6016\571235199.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_positive['d'] = dataframe_positive['z'].apply(calculate_d)
C:\Users\giorg\AppData\Local\Temp\ipykernel_6016\571235199.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_negative['d'] = dataframe_negative['z'].apply(calculate_d)


In [471]:
# count unique values of dataframe_positive['d']
dataframe_positive['d'].value_counts()

8.967     75804
9.092     72818
8.842     72449
9.217     62931
8.717     62383
8.592     51092
9.342     50351
8.467     43516
8.092     42891
11.092    42877
11.217    42849
8.217     42802
10.967    42277
11.342    42066
11.467    41389
8.342     41109
10.842    41082
11.592    40948
7.967     40615
11.717    39992
11.842    39495
12.092    39341
11.967    39226
12.342    39032
10.717    39014
9.467     38872
12.217    38849
12.467    38723
12.717    38696
12.592    38306
12.967    37869
12.842    37858
13.092    37384
13.217    37297
13.342    36833
10.592    36709
13.467    36192
13.592    35029
13.717    34703
10.467    33376
7.842     33133
13.842    33108
13.967    32087
9.592     30937
10.342    30838
10.217    27901
9.717     26436
10.092    25767
9.842     24897
9.967     24373
7.717     23127
7.592     12697
14.092     8500
7.467      4843
7.342      1107
7.217       151
7.092        13
6.967         1
Name: d, dtype: int64

In [472]:
dataframe_positive[dataframe_positive['d'] == 7.842]

,atom,x,y,z,frame,d
1072,HW,14.908,10.602,7.741,1.0,7.842
1314,HW,10.222,-7.668,7.748,1.0,7.842
1381,HW,25.226,12.427,7.814,1.0,7.842
1551,HW,-6.787,13.743,7.823,1.0,7.842
2020,HW,-5.269,-6.130,7.732,1.0,7.842
...,...,...,...,...,...,...
10127381,HW,-17.886,16.879,7.786,3000.0,7.842
10127798,HW,-7.762,-12.116,7.743,3000.0,7.842
10127818,HW,-7.200,-6.170,7.780,3000.0,7.842
10127866,HW,-10.644,13.640,7.748,3000.0,7.842


In [473]:
# Raggruppamento per frame e d, calcolo del conteggio
grouped_positive = dataframe_positive.groupby(['frame', 'd']).size().reset_index(name='count')
grouped_negative = dataframe_negative.groupby(['frame', 'd']).size().reset_index(name='count')

In [474]:
grouped_negative.tail()

,frame,d,count
161706,3000.0,13.592,12
161707,3000.0,13.717,6
161708,3000.0,13.842,11
161709,3000.0,13.967,13
161710,3000.0,14.092,3


In [475]:
grouped_positive.head()

,frame,d,count
0,1.0,7.467,3
1,1.0,7.592,1
2,1.0,7.717,7
3,1.0,7.842,9
4,1.0,7.967,11


In [476]:
# Creazione del dataframe di riferimento con tutte le combinazioni di frame e d
frames1 = np.arange(1.0, 3001.0, 1.0)
d_values1 = np.arange(5.842, 14.125, 0.125)
df_reference = pd.DataFrame([(frame, d) for frame in frames1 for d in d_values1], columns=['frame', 'd'])

In [477]:
grouped_negative_fill = pd.merge(grouped_negative, df_reference, how='right', on=['frame', 'd'])
grouped_positive_fill = pd.merge(grouped_positive, df_reference, how='right', on=['frame', 'd'])
grouped_negative_fill['count'].fillna(0, inplace=True)
grouped_positive_fill['count'].fillna(0, inplace=True)

In [478]:
grouped_positive_fill.head()

,frame,d,count
0,1.0,5.842,0.0
1,1.0,5.967,0.0
2,1.0,6.092,0.0
3,1.0,6.217,0.0
4,1.0,6.342,0.0


In [479]:
positive_negative = pd.merge(grouped_positive_fill, grouped_negative_fill, how='inner', on=['frame', 'd'])
positive_negative_grouped = positive_negative.groupby('d')[['count_x', 'count_y']].mean()
positive_negative_grouped

# Rinomina delle colonne del nuovo dataframe
positive_negative_grouped.columns = ['Average N of H (positive z)', 'Average N of H (negative z)']

# Creazione degli indici come valori da 7.625 a 14
d_values = np.arange(5.842, 14.125, 0.125)
# subtract 1.78271213799185 to every value of d_values
d_values -= 5.842
positive_negative_grouped.index = d_values
positive_negative_grouped

,Average N of H (positive z),Average N of H (negative z)
0.000,0.000000,0.000000
0.125,0.000000,0.000000
0.250,0.000000,0.000000
0.375,0.000000,0.000000
0.500,0.000000,0.000000
...,...,...
7.750,11.676333,8.631667
7.875,11.567667,8.126333
8.000,11.036000,7.503333
8.125,10.695667,7.004333


In [480]:
def calculate_positive_density(positive_density):
    positive_density /= 2
    positive_density *= 18
    positive_density /= 0.196079405
    positive_density /= 6.02214e23
    positive_density *= 1e21
    return positive_density

positive_negative_grouped['Average Density (positive z)'] = positive_negative_grouped['Average N of H (positive z)'].apply(calculate_positive_density)
positive_negative_grouped['Average Density (negative z)'] = positive_negative_grouped['Average N of H (negative z)'].apply(calculate_positive_density)

In [481]:
positive_negative_grouped

,Average N of H (positive z),Average N of H (negative z),Average Density (positive z),Average Density (negative z)
0.000,0.000000,0.000000,0.000000,0.000000
0.125,0.000000,0.000000,0.000000,0.000000
0.250,0.000000,0.000000,0.000000,0.000000
0.375,0.000000,0.000000,0.000000,0.000000
0.500,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...
7.750,11.676333,8.631667,0.889951,0.657892
7.875,11.567667,8.126333,0.881669,0.619376
8.000,11.036000,7.503333,0.841146,0.571892
8.125,10.695667,7.004333,0.815206,0.533859


In [482]:
# salva il dataset come file .csv
positive_negative_grouped.to_csv("./output/HNear_" + files_pdb[x].rstrip(".pdb") + ".csv")

# NaCl

In [364]:
# mantain only rows with atom == 'HW'
dataframe_NaCl = dataframe[dataframe['atom'].isin(['CX', 'NX'])]
dataframe_NaCl.head()

,atom,x,y,z,frame
196,NX,9.591,-18.772,3.133,1.0
197,NX,15.141,-18.825,2.897,1.0
198,NX,12.359,-16.144,2.942,1.0
199,NX,17.950,-15.855,3.026,1.0
200,NX,9.638,-13.067,2.663,1.0


In [365]:
dataframe_NaCl.tail()

,atom,x,y,z,frame
10126261,CX,9.639,15.108,-5.843,3000.0
10126262,CX,15.185,15.227,-5.720,3000.0
10126263,CX,12.384,18.097,-5.972,3000.0
10126264,CX,18.089,17.977,-5.595,3000.0
10126265,CX,9.521,9.483,-5.969,3000.0


In [366]:
dataframe_NaCl["atom"].value_counts(dropna=False)

NX    1176000
CX    1176000
Name: atom, dtype: int64

Splitting in positive and negative values

In [367]:
dataframe_positive = dataframe_NaCl.loc[dataframe['z'] > 0]
dataframe_negative = dataframe_NaCl.loc[dataframe['z'] < 0]

Effettua calcoli:
1. Trova l'indice di distanza da assegnare a ogni idrogeno
2. elimina gli idrogeni più distanti di 14

In [368]:
def calculate_d(z):
    d_values = np.arange(0.125, 14.125, 0.125)
    for d in d_values:
        if abs(z) <= d:
            return d
    return d_values[-1]

# Aggiunta della colonna "d" nei dataframe
dataframe_positive['d'] = dataframe_positive['z'].apply(calculate_d)
dataframe_negative['d'] = dataframe_negative['z'].apply(calculate_d)
dataframe_positive = dataframe_positive.loc[abs(dataframe_positive['z']) <= 14]
dataframe_negative = dataframe_negative.loc[abs(dataframe_negative['z']) <= 14]

C:\Users\giorg\AppData\Local\Temp\ipykernel_6016\2096704374.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_positive['d'] = dataframe_positive['z'].apply(calculate_d)
C:\Users\giorg\AppData\Local\Temp\ipykernel_6016\2096704374.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_negative['d'] = dataframe_negative['z'].apply(calculate_d)


In [369]:
dataframe_positive[dataframe_positive['d'] == 14]

,atom,x,y,z,frame,d


In [370]:
dataframe_positive.head()

,atom,x,y,z,frame,d
196,NX,9.591,-18.772,3.133,1.0,3.250
197,NX,15.141,-18.825,2.897,1.0,3.000
198,NX,12.359,-16.144,2.942,1.0,3.000
199,NX,17.950,-15.855,3.026,1.0,3.125
200,NX,9.638,-13.067,2.663,1.0,2.750


In [371]:
# Raggruppamento per frame e d, calcolo del conteggio
grouped_positive_nx = dataframe_positive[dataframe_positive["atom"] == "NX"].groupby(['frame', 'd']).size().reset_index(name='count')
grouped_positive_cx = dataframe_positive[dataframe_positive["atom"] == "CX"].groupby(['frame', 'd']).size().reset_index(name='count')
grouped_negative_nx = dataframe_negative[dataframe_negative["atom"] == "NX"].groupby(['frame', 'd']).size().reset_index(name='count')
grouped_negative_cx = dataframe_negative[dataframe_negative["atom"] == "CX"].groupby(['frame', 'd']).size().reset_index(name='count')


In [372]:
grouped_positive_nx.head()

,frame,d,count
0,1.0,2.750,5
1,1.0,2.875,25
2,1.0,3.000,47
3,1.0,3.125,15
4,1.0,3.250,5


In [373]:
grouped_negative_nx.head()

,frame,d,count
0,1.0,2.750,5
1,1.0,2.875,37
2,1.0,3.000,35
3,1.0,3.125,19
4,1.0,3.250,2


In [374]:
# Creazione del dataframe di riferimento con tutte le combinazioni di frame e d
frames1 = np.arange(1.0, 3001.0, 1.0)
d_values1 = np.arange(.125, 14.125, 0.125)
df_reference = pd.DataFrame([(frame, d) for frame in frames1 for d in d_values1], columns=['frame', 'd'])

In [375]:
grouped_positive_nx['d'].value_counts()

6.125    3000
5.750    3000
3.000    3000
3.125    3000
2.875    3000
6.000    3000
5.875    3000
5.625    3000
2.750    2999
6.250    2978
3.250    2840
6.375    2322
5.500    1981
6.500     978
3.375     744
2.625     288
6.625     261
5.375     131
6.750      67
3.500      47
6.875      17
3.625       3
7.000       2
Name: d, dtype: int64

In [376]:
grouped_negative_fill_nx = pd.merge(grouped_negative_nx, df_reference, how='right', on=['frame', 'd'])
grouped_negative_fill_cx = pd.merge(grouped_negative_cx, df_reference, how='right', on=['frame', 'd'])

grouped_positive_fill_nx = pd.merge(grouped_positive_nx, df_reference, how='right', on=['frame', 'd'])
grouped_positive_fill_cx = pd.merge(grouped_positive_cx, df_reference, how='right', on=['frame', 'd'])

grouped_negative_fill_nx['count'].fillna(0, inplace=True)
grouped_positive_fill_nx['count'].fillna(0, inplace=True)

grouped_negative_fill_cx['count'].fillna(0, inplace=True)
grouped_positive_fill_cx['count'].fillna(0, inplace=True)

In [377]:
grouped_negative_fill_nx.head()

,frame,d,count
0,1.0,0.125,0.0
1,1.0,0.250,0.0
2,1.0,0.375,0.0
3,1.0,0.500,0.0
4,1.0,0.625,0.0


In [378]:
positive_negative_nx = pd.merge(grouped_positive_fill_nx, grouped_negative_fill_nx, how='inner', on=['frame', 'd'])
positive_negative_grouped_nx = positive_negative_nx.groupby('d')[['count_x', 'count_y']].mean()
positive_negative_grouped_nx

# Rinomina delle colonne del nuovo dataframe
positive_negative_grouped_nx.columns = ['Average N of Na (positive z)', 'Average N of Na (negative z)']

positive_negative_cx = pd.merge(grouped_positive_fill_cx, grouped_negative_fill_cx, how='inner', on=['frame', 'd'])
positive_negative_grouped_cx = positive_negative_cx.groupby('d')[['count_x', 'count_y']].mean()
positive_negative_grouped_cx

# Rinomina delle colonne del nuovo dataframe
positive_negative_grouped_cx.columns = ['Average N of Cl (positive z)', 'Average N of Cl (negative z)']

# Creazione degli indici come valori da 7.625 a 14
# d_values = np.arange(0.125, 14.125, 0.125)
# # subtract 1.78271213799185 to every value of d_values
# d_values -= 5.84228786200815
# positive_negative_grouped.index = d_values
# positive_negative_grouped

In [379]:
positive_negative_grouped_cx

,Average N of Cl (positive z),Average N of Cl (negative z)
d,,
0.125,0.0,0.0
0.250,0.0,0.0
0.375,0.0,0.0
0.500,0.0,0.0
0.625,0.0,0.0
...,...,...
13.500,0.0,0.0
13.625,0.0,0.0
13.750,0.0,0.0


In [380]:
positive_negative = pd.merge(positive_negative_grouped_cx, positive_negative_grouped_nx, how='inner', on=['d'])
positive_negative

,Average N of Cl (positive z),Average N of Cl (negative z),Average N of Na (positive z),Average N of Na (negative z)
d,,,,
0.125,0.0,0.0,0.0,0.0
0.250,0.0,0.0,0.0,0.0
0.375,0.0,0.0,0.0,0.0
0.500,0.0,0.0,0.0,0.0
0.625,0.0,0.0,0.0,0.0
...,...,...,...,...
13.500,0.0,0.0,0.0,0.0
13.625,0.0,0.0,0.0,0.0
13.750,0.0,0.0,0.0,0.0


In [381]:
# salva il dataset come file .csv
positive_negative.to_csv("./output/NaCl_" + files_pdb[x].rstrip(".pdb") + ".csv")